In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.datasets import load_sample_image

china = load_sample_image("china.jpg")/255
flower = load_sample_image("flower.jpg")/255
images = np.array([china,flower])

In [3]:
class ResidualLayer(keras.layers.Layer):

    def __init__(self, filters, stride, activation, **kwargs):
        super().__init__(**kwargs)
        self.activation = activation
        self.layers = [
            keras.layers.Conv2D(filters, 3, stride, padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, 1, padding='same', use_bias=False),
            keras.layers.BatchNormalization(),   
        ]
        self.skip = [

        ]
        if stride > 1:
            self.skip.append(keras.layers.Conv2D(filters, 1, stride, padding= 'same', use_bias=False))
            self.skip.append(keras.layers.BatchNormalization())
    
    def call(self,inputs):
        z = inputs
        x = inputs
        for layer in self.layers:
            x = layer(x)
        for layer in self.skip:
            z = layer(z)
        x = self.activation(x+z)
        return x

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,2,input_shape=[224,224,3],padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU()),
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))
prev = 64
for i in [64,64,64,128,128,128,128,256,256,256,256,512,512,512]:
    strides = 1 if prev == i else 2
    model.add(ResidualLayer(i,strides,keras.layers.ReLU()))
    prev = i
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())   
model.add(keras.layers.Dense(10,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer="nadam", metrics=['accuracy'])

In [7]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)


Dl Completed...: 100%|██████████| 1/1 [00:16<00:00, 16.90s/ url]


Dataset tf_flowers downloaded and prepared to C:\Users\marksan\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.
